# 使用情境 - 1

你現在是一個作文批改老師，你想要使用所學的 Langchain 知識來幫助你批改作文，建立一個作文批改的模型，輸入一篇作文，自訂輸出格式以滿足 cell 內題目的需求。

你會需要:

    1. 定義 schema，控制模型輸出
    2. 建立 output parser
    3. 定義輸入的模板
    4. 利用 LCEL 的語法建立一個基本的 chain
    5. 將文章 input 到 chain 來產生 output
    
- 範例如下：
    - 輸入：
        ```python
        input_ = {
            "article": "愛迪生說：「友誼能增進快樂，減少痛苦」，可見朋友有多重要。朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走；朋友像一位溫柔的老師，當我功課不會寫時，他會很用心的教導我；朋友像一顆開心果，當我難過或傷心時，講笑話給我聽；朋友像一個太陽，當我遇到困難時，覺得人生黑暗的時候，他會溫暖我的心。猶記一年級的時候，我有一位很要好的朋友，她喜歡運動和打球，而我也很喜歡運動，下課的時候，我常常跟她一起玩。她很大方，也很有禮貌，又喜歡幫老師做事，人緣好，所以大家都選她當班長。在她身上我看到很多優點，也學到很多，有了她的陪伴，所以每天在學校都過得很開心。有一次我在操場上跌倒流血，倒在地上爬不起來，是她扶我起來，陪我到健康中心。接下來那幾天，她幫我打掃、交作業，這讓我深深體會好朋友是多麼重要。雖然我們升上中年級後沒有在同一班，但我們還是思念著對方，下課時常常聚在一起，我的心中仍將她視為最好的朋友。如果一個人沒有朋友，就找不到陪她一起玩樂和訴苦的人，這樣就會很孤單。朋友是一生中絕對不能少的，缺少了他，我可能每天心情不好而感到孤獨寂寞。有了好朋友，是一件很幸福的事，也要懂得珍惜，建立的友誼才能長久。"
        }
        ```
    - 輸出
        ```python
            {
                '切題程度': 8, 
                '文筆': 7, 
                '創意': 6, 
                '通順': 9
            }
        ```

In [1]:
import warnings
from langchain.prompts import ChatPromptTemplate
from langchain.chains.openai_functions.base import convert_to_openai_function
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from langchain_setup import ChatOpenAI

In [2]:
"""
======================================================
cousre practice: 
用課程先前所學，建立一個LLM模型來批改文章，並以特定框架output

給定一篇文章 `article`，讓模型給出以下四個分數：

1. 依照切題程度給予評分，分數須介於 1~10分之間
2. 依照內容深度給予評分，分數須介於 1~10分之間
3. 依照創意程度給予評分，分數須介於 1~10分之間
4. 依照通順程度給予評分，分數須介於 1~10分之間

===================================================
"""

input_ = {"article": "愛迪生說：「友誼能增進快樂，減少痛苦」，可見朋友有多重要。朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走；朋友像一位溫柔的老師，當我功課不會寫時，他會很用心的教導我；朋友像一顆開心果，當我難過或傷心時，講笑話給我聽；朋友像一個太陽，當我遇到困難時，覺得人生黑暗的時候，他會溫暖我的心。猶記一年級的時候，我有一位很要好的朋友，她喜歡運動和打球，而我也很喜歡運動，下課的時候，我常常跟她一起玩。她很大方，也很有禮貌，又喜歡幫老師做事，人緣好，所以大家都選她當班長。在她身上我看到很多優點，也學到很多，有了她的陪伴，所以每天在學校都過得很開心。有一次我在操場上跌倒流血，倒在地上爬不起來，是她扶我起來，陪我到健康中心。接下來那幾天，她幫我打掃、交作業，這讓我深深體會好朋友是多麼重要。雖然我們升上中年級後沒有在同一班，但我們還是思念著對方，下課時常常聚在一起，我的心中仍將她視為最好的朋友。如果一個人沒有朋友，就找不到陪她一起玩樂和訴苦的人，這樣就會很孤單。朋友是一生中絕對不能少的，缺少了他，我可能每天心情不好而感到孤獨寂寞。有了好朋友，是一件很幸福的事，也要懂得珍惜，建立的友誼才能長久。"}

# ======================================================
# Define schema,validation and postprocessing
# ======================================================
class Article(BaseModel):
    """批改作文的老師"""
    
    # TODO: 定義模型的輸出格式，包含切題程度 (relevance)、內容深度 (writing)、創意程度 (creativity)、通順程度 (coherence)。
    relevance: int =
    writing: int = 
    creativity: int = 
    coherence: int = 

    # 確認成績的範圍為1~10
    @root_validator() 
    def validate_values(cls, values: dict) -> dict:
        for key in values.keys():
            if not (1 <= values[key] <= 10):
                # TODO: 若分數的範圍為不在 1~10 之間，則請更改輸出使其符合範圍
            
        return values

# build ouput parser
output_parser = PydanticOutputFunctionsParser(pydantic_schema=Article)

# ======================================================
# Define input template
# ======================================================
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一名國文教師，你現在要批改學生的作文，題目為朋友，評分標準需為以下四項：切題、文筆、創意、通順。"),
        ("human", "你好，請問可以批改我的作文嗎。"),
        ("ai", "好的，請給我看你的文章。"),
        # TODO: 加入對話，讓模型可以吃 input 中的 `article`

    ],
)

# ======================================================
# Chain it together
# ======================================================

model = ChatOpenAI(temperature=0)

# TODO: 將上面定義模型輸出的 class 轉換為 OpenAI function 的格式
function_kwargs = 

# TODO: 建立 OpenAI function calling 的 kwargs
openaifn_kwargs: dict = 

# TODO: 透過 LCEL 語法建立基礎的 runnable（使用 `|`），別忘記你的 openaifn_kwargs
runnable = 

output = runnable.invoke(input_)
print(
    {
        "切題程度": output.relevance,
        "文筆": output.writing,
        "創意": output.creativity,
        "通順": output.coherence,
    }
)

{'切題程度': 8, '文筆': 7, '創意': 6, '通順': 9}


# 使用情境 - 2

請同學利用 RunnableMap 來同時處理兩種不同種類的問題，並建立兩個 runnable 來回答，一個 runnable 用來介紹景點，另一個 runnable 則用來介紹食物，最後用 RunnablePassthrough 在保留 input 的同時並新增 output
        
    
  - 範例如下：
      - 輸入：
        ```
        {"place": "台灣"}
        ```
      - 輸出：

        ```python
        outputs: {
            'attraction introduction': Attraction_descriptor(attraction='台北101', introduction='台北101是台灣最著名的地標之一，也是世界上最高的摩天大樓之一。它擁有觀景台、購物中心、餐廳和展覽館等設施，遊客可以欣賞到壯觀的城市全景。此外，每年跨年夜時，台北101的煙火秀也是台灣最受矚目的活動之一。'), 
            'food_introduction': food_descriptor(food='小籠包', introduction='小籠包是台灣的特色美食之一，它是一種以豬肉為餡料，以麵粉皮包裹而成的小型餃子。小籠包的特色在於其餡料鮮嫩多汁，皮薄餡多，每一口都能品嚐到豐富的肉汁。它通常搭配黑醋和生薑一起食用，酸甜的醋和辣味的生薑能夠提升小籠包的風味。小籠包是台灣的代表性美食之一，絕對值得一試！')
        }
        ```
    

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_setup import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema, PydanticOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableMap, RunnableBranch, RunnablePassthrough

# Scenario: Developing a Question Answering Pipeline

# Components Initialization

# [Attraction runnable] Prompt
attraction_prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "你是一位智能導遊機器人。請根據輸入的疑問滿足旅客的要求。"),
    # TODO: 建立對話，讓模型可以藉由 input 中的地點，介紹一個景點給你
],
)

# [Attraction runnable] Language model 
attraction_model = ChatOpenAI(temperature=0)


# [Attraction runnable] Output formatting
class AttractionDescriptor(BaseModel):
    """介紹景點"""

    # TODO: 定義模型的輸出，需要模型給出一個景點並介紹
    attraction: str = 
    introduction: str = 

attraction_output_parser = PydanticOutputFunctionsParser(pydantic_schema=AttractionDescriptor)

# TODO: 將 class 轉為 openai_function, 並定義 kwargs
attraction_function_kwargs = 
attraction_openaifn_kwargs = 

# [Attraction runnable] Runnable
# TODO: Chain the components - Prompt, Model, Output Parser with the OpenAI function
attraction_runnable = 


# [Food runnable] Prompt
food_prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "你是一位智能導遊機器人。請根據輸入的疑問滿足旅客的要求。"),
    # TODO: 建立對話模板，讓模型介紹一個當地美食給你
],
)

# [Food runnable] Language model
food_model = ChatOpenAI(temperature=0)

# [Food runnable] Output formatting
class FoodDescriptor(BaseModel):
    """介紹食物"""
    
    # TODO: 定義模型的輸出，需要模型給出一個美食並介紹
    food: str = 
    introduction: str = 

food_output_parser = PydanticOutputFunctionsParser(pydantic_schema=FoodDescriptor)

# TODO: 將上面定義模型輸出的 class 轉換為 OpenAI function 的格式。並建立 OpenAI function calling 的 kwargs
food_function_kwargs = 
food_openaifn_kwargs = 

# [Food runnable] Runnable
# TODO: Chain the components - Prompt, Model, Output Parser with the OpenAI function
food_runnable = 


# Combine different model output using RunnableMap
branched_runnable = RunnableMap({
    # TODO: 用 RunnalbelMap 將 input 同時給兩個不同的 chain
})

inputs = {"place": "台灣"}

add_by_pass = # TODO: 用 RunnablePassthrough 在保留 input 的同時並新增 output
outputs = add_by_pass.invoke(inputs)
print(f'input: {outputs["place"]}\noutputs: {outputs["output"]}')

input: 台灣
outputs: {'attraction introduction': Attraction_descriptor(attraction='台北101', introduction='台北101是台灣最著名的地標之一，也是世界上最高的摩天大樓之一。它擁有觀景台、購物中心、餐廳和展覽館等設施，遊客可以欣賞到壯觀的城市全景。此外，每年跨年夜時，台北101的煙火秀也是台灣最受矚目的活動之一。'), 'food_introduction': food_descriptor(food='小籠包', introduction='小籠包是台灣的特色美食之一，它是一種以豬肉為餡料，以麵粉皮包裹而成的小型餃子。小籠包的特色在於其餡料鮮嫩多汁，皮薄餡多，每一口都能品嚐到豐富的肉汁。它通常搭配黑醋和生薑一起食用，酸甜的醋和辣味的生薑能夠提升小籠包的風味。小籠包是台灣的代表性美食之一，絕對值得一試！')}
